# RBC model


> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
# 1. We import the packages
import numpy as np
from scipy import optimize
import sympy as sm
import pandas as pd

# Autoreloads modules when code is run
%load_ext autoreload
%autoreload 2

# We import the local modules
import modelproject

# Model description

**RBC Model Setup - Assumptions** 

- Time is discrete,  $t\in\{0,1,2, \dots\}$.

- Large numbers of identical price-taking firms.

- Large numbers of indentical price-taking households.

- Households are infinitely lived.

**The benevolent social planner solves the optimal growth problem:**
$c_t$ is consumption, $l_t$ is leisure for the household, $n_t$ is hours worked, $l_t$ is leisure, and $k_t$ is capital.

\begin{aligned}
    \max_{\{c_t,n_t,k_{t+1}\}^{\infty}_{t=0}} &\mathbb{E}_0 \sum^{\infty}_{t=0} \beta^t u\left(c_t, n_t\right), \quad \beta \in (0,1) \\

    &\text{s.t.}\\
    
\end{aligned}

Workhours-leisure constraint: 

\begin{aligned}
n_t + l_t = 1
\end{aligned} \\

Technology:

\begin{aligned}

 y_t &= A_t F(k_t,n_t) 

\end{aligned} \\

The resource constraint:

\begin{aligned}

 y_t &=c_t+i_t  

\end{aligned} \\

Capital akkumulation:
\begin{aligned}
k_{t+1} &=i_t+(1-\delta)k_t 
\end{aligned}

The production function is Cobb-Douglas:
\begin{aligned}
F(k_t,n_t) = k_t^\alpha n_t^{1-\alpha}
\end{aligned}

The utility function:

\begin{aligned}
u(c_t,n_t)=log(c_t)+\psi l_t
\end{aligned}

The Lagrangian:

\begin{aligned}
L &= \mathbb{E}_0\left[ \sum^{\infty}_{t=0} \beta^t u\left(c_t, n_t\right) + \lambda_t \beta^t (A_t F(k_t,n_t)+(1-\delta)k_t-c_t-k_{t+1}+\omega_t\beta^t(1-l_t-n_t))\right]

\end{aligned}

**First-Order Conditions** 




## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

### Linear approxmiations

Before we can numerically solve the model then we need to *log linearize* around the steady state. This is because the RBC model is a non-linear model then we have to linerazise around the Steady State. Otherwise, it will be harder to solve numerically.

In addition, we *log linearize* because it is easier to interpret percentages instead of units. Before we move on, the theory behind linear approxmiation is introduced to make sure the process is clear. The theory is from Alexandre Kohlhas' Macroeconomics III slides.

The total derivative of $f(x)$ at the steady state level for a vector of variables $x = x_t^*$:

\begin{aligned}
    \text{df}(x_t) = \sum^n_{j=1} \dfrac{\partial f(x_t)}{\partial x_j}_{|x_t=x^*} dx_j
\end{aligned}

So, total differentiation is approxmiately a Taylor approximation of first-order.

### Log-Linearization

The change in our variables is given by:
\begin{aligned}
    f(x_t) &= f(y_t, c_t, i_t, n_t, l_t, k_t)\\
    &= f(\exp^{\log\left(y_t\right)}, \exp^{\log\left(c_t\right)}, \exp^{\log\left(i_t\right)}, \exp^{\log\left(n_t\right)}, \exp^{\log\left(l_t\right)}, \exp^{\log\left(k_t\right)})
\end{aligned}

\begin{aligned}
    \begin{pmatrix}
        \psi c_t\\
        \dfrac{1}{c_t}\\
        y_t\\
        y_t\\
        k_{t+1}\\
        1
    \end{pmatrix} = \begin{pmatrix} 
        (1 - \alpha) z_t \left(\dfrac{k_t}{n_t}\right)^{\alpha}\\
        \beta \mathbb{E}_t \left[\dfrac{1}{c_{t+1}}\left(\alpha A_{t+1} \left(\dfrac{k_{t+1}}{n_{t+1}}\right)^{\alpha-1}\right) + (1 - \delta)\right]\\
        A_t k_t^{\alpha}n_t^{1-\alpha}\\
        y_t = c_t + i_t\\
        (1-\delta)k_t + i_t\\
        1 + l_t + n_t
    \end{pmatrix}
\end{aligned}

To numerically solve the RBC model, we define a system of variables:

\begin{aligned}
    x_t &= \begin{pmatrix} 
    \text{Output}\\
    \text{Consumption}\\
    \text{Investment}\\
    \text{Labour}\\
    \text{Leisure}\\
    \text{Capital}\\
    \end{pmatrix}\\ 
    &= \begin{pmatrix}
    y_t \\
    c_t\\
    i_t\\
    n_t\\
    l_t\\
    k_t\\
    \end{pmatrix}
\end{aligned}

Next, we define a paramter space, $\Theta$:

\begin{aligned}
    \Theta = \begin{pmatrix} 
        \text{Discount rate}\\
        \text{Marginal disutility of labour}\\
        \text{}
        \text{Capital-share of output}\\
    \end{pmatrix}
\end{aligned}

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion